<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import numpy as np
import pandas as pd

import re
import time

import requests
from bs4 import BeautifulSoup

from IPython.display import display
from ipywidgets import FloatProgress

In [5]:
t0 = time.time()

# url
url = 'http://www.imdb.com/chart/top?ref_=nv_mv_250_6'

# get soup
result = requests.get(url)
c = result.content
soup = BeautifulSoup(c,"lxml")

# Create empty lists to append the extracted data.
moviename = []
cast = []
description = []
rating = []
ratingoutof = []
year = []
genre = []
movielength = []
rot_audscore = []
rot_avgrating = []
rot_users = []

# Extracting the required data from the html soup.
rgx = re.compile('[%s]' % '()')
f = FloatProgress(min=0, max=200)
display(f)

# summary
summary = soup.find('div',{'class':'article'})

for row, i in zip(summary.find('table').findAll('tr'),
                  range(len(summary.find('table').findAll('tr')))):
    
    # get year
    for sitem in row.findAll('span', {'class': 'secondaryInfo'}):
        s = sitem.find(text=True)
        year.append(rgx.sub("", s))
        
    # get ratings
    for ritem in row.findAll('td', {'class': "ratingColumn imdbRating"}):
        for iget in ritem.findAll('strong'):
            rating.append(iget.find(text=True))
            ratingoutof.append(iget.get('title').split(' ', 4)[3])

    # tmdb items             
    for item in row.findAll('td',{'class':'titleColumn'}):
        for href in item.findAll('a',href=True):
            moviename.append(href.find(text=True))
            rurl = 'https://www.rottentomatoes.com/m/'+ href.find(text=True)

            # get rotten url
            try:
                rresult = requests.get(rurl)
            except requests.exceptions.ConnectionError:
                status_code = "Connection refused"

            # rc
            rc = rresult.content
            rsoup = BeautifulSoup(rc)
                
            # 
            try:
                raud = rsoup.find('div', {'class':'meter-value'})
                raud = raud.find('span',{'class':'superPageFontColor'})
                raud = raud.text
                rot_audscore.append(raud)

                ravg = rsoup.find('div',{'class':'audience-info hidden-xs superPageFontColor'})
                ravg = ravg.find('div')
                ravg = ravg.contents[2].strip()
                rot_avgrating.append(ravg)

                ru = rsoup.find('div', {'class':'audience-info hidden-xs superPageFontColor'})
                ru = ru.contents[3].contents[2].strip()
                rot_users.append(ru)

            except AttributeError:
                rot_audscore.append("")
                rot_avgrating.append("")
                rot_users.append("")

            cast.append(href.get('title'))
            imdb = "http://www.imdb.com" + href.get('href')

            try:
                iresult = requests.get(imdb)
                ic = iresult.content
                isoup = BeautifulSoup(ic)
                #print(isoup,file=open('isoup.html','w'))
                
                # desc
                #
                # <div class="summary_text">
                #    The early life and career of Vito Corleone 
                # </div>
                #
                desc = isoup.find('div', {'class':'summary_text'})
                desc = desc.find(text=True).strip()
                description.append(desc)


                # genre
                # <a href="/search/keyword?keywords=mafia"> <span class="itemprop">
                # mafia</span></a><span>|</span>
                #
                # genre = mafia
                gen = isoup.find('span',{'class':'itemprop'})
                gen = gen.find(text=True)
                genre.append(gen)
                

                # duration
                #
                # <time datetime="PT175M">
                #        2h 55min
                #   </time>
                #
                dt = isoup.find('time')
                dt = dt.find(text=True).strip()
                movielength.append(dt)


            except requests.exceptions.ConnectionError:
                description.append("")
                genre.append("")
                movielength.append("")
        # sleep  
        time.sleep(.1)
        f.value = i

t1 = time.time() - t0
print('Time taken {:.0f} min {:.0f} secs: '.format(*divmod(t1,60)))

FloatProgress(value=0.0, max=200.0)

Time taken 10 min 22 secs: 


In [7]:
# create dataframe
imdb = pd.DataFrame({'rank': range(1, len(year)+1),
                     'moviename': moviename,
                     'year': year,
                     'imdb_rating': rating,
                     'cast': cast,
                     'description': description,
                     'genre': genre,
                     'movielength': movielength,
                     'cast': cast,
                     'imdb_ratingbasedon': ratingoutof,
                     'tomatoes_audscore': rot_audscore,
                     'tomatoes_rating': rot_avgrating,
                     'tomatoes_ratingbasedon': rot_users})

print(imdb.shape)
imdb.head()

(250, 12)


,rank,moviename,year,imdb_rating,cast,description,genre,movielength,imdb_ratingbasedon,tomatoes_audscore,tomatoes_rating,tomatoes_ratingbasedon
0,1,The Shawshank Redemption,1994,9.2,"Frank Darabont (dir.), Tim Robbins, Morgan Fre...",Two imprisoned men bond over a number of years...,wrongful imprisonment,2h 22min,"2,128,423",,,
1,2,The Godfather,1972,9.1,"Francis Ford Coppola (dir.), Marlon Brando, Al...",The aging patriarch of an organized crime dyna...,mafia,2h 55min,"1,461,435",,,
2,3,The Godfather: Part II,1974,9.0,"Francis Ford Coppola (dir.), Al Pacino, Robert...",The early life and career of Vito Corleone in ...,revenge,3h 22min,"1,016,521",,,
3,4,The Dark Knight,2008,9.0,"Christopher Nolan (dir.), Christian Bale, Heat...",When the menace known as The Joker emerges fro...,dc comics,2h 32min,"2,093,092",,,
4,5,12 Angry Men,1957,8.9,"Sidney Lumet (dir.), Henry Fonda, Lee J. Cobb",A jury holdout attempts to prevent a miscarria...,jury,1h 36min,"606,526",,,


In [8]:
!open .